<a href="https://colab.research.google.com/github/shouryan01/COVID-19-Tracker/blob/master/covid19_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#IMPORT LIBRARIES

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
%matplotlib inline
import seaborn as sns

sns.set()
sns.set_style("whitegrid")
custom_style = {
            'grid.color': '0.8',
            'grid.linestyle': '--',
            'grid.linewidth': 0.5,
}
sns.set_style(custom_style)

print("Done!")

Done!


In [32]:
#LOAD DATA

GLOBAL_CASES_SOURCE      = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
GLOBAL_DEATHS_SOURCE     = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
GLOBAL_RECOVERIES_SOURCE = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

global_cases      = pd.read_csv(GLOBAL_CASES_SOURCE)
global_deaths     = pd.read_csv(GLOBAL_DEATHS_SOURCE)
global_recoveries = pd.read_csv(GLOBAL_RECOVERIES_SOURCE)
datasets = [global_cases, global_deaths, global_recoveries]

date_yesterday = pd.to_datetime(global_cases.columns[-1]).date()
date_today     = date_yesterday + datetime.timedelta(days=1)

print('\nData was last updated on ' + str(date_today) + '.' + ' Latest data available for ' + str(date_yesterday))


Data was last updated on 2020-08-16. Latest data available for 2020-08-15


In [33]:
#CLEAN DATA

def drop_negatives(df):
    idx = df[df.iloc[:, -1] < 0].index.tolist()
    for i in idx:
        df.drop([i], inplace = True)
    return df.reset_index(drop = True)

datasets = [drop_negatives(i) for i in datasets]

for i in range(3):
    datasets[i][['Province/State']] = datasets[i][['Province/State']].fillna('')
    datasets[i].fillna(0, inplace=True)
    datasets[i].columns = datasets[i].columns[:4].tolist() + \
                          [pd.to_datetime(datasets[i].columns[j]).date() for j in range(4, len(datasets[i].columns))]



print("Done!")

Done!


In [40]:
country_datasets = []
for i in range(3):
    total = datasets[i].groupby('Country/Region').sum()
    total.rename(index={'US':'United States'}, inplace = True)
    country_datasets.append(total)

print("Done!")

Done!


In [35]:
#INSTALL country_converter

!pip install country_converter --upgrade

Requirement already up-to-date: country_converter in /usr/local/lib/python3.6/dist-packages (0.6.7)


In [47]:
#LABEL CONTINENT

import country_converter as coco
for dataset in country_datasets:
    continent = coco.convert(names=dataset.index.tolist(), to='Continent')
    dataset['Continent'] = continent

print("Done!")

Done!


In [55]:
print('Global Cases      : {}'.format(datasets[0].sum()[date_yesterday]))
print('Global Deaths     : {}'.format(datasets[1].sum()[date_yesterday]))
print('Global Recoveries : {}'.format(datasets[2].sum()[date_yesterday]))


Global Cases      : 21459699
Global Deaths     : 771063
Global Recoveries : 13445842
